In [2]:
%pip install \
    transformers \
    datasets \
    evaluate \
    rouge_score\
    loralib \
    bitsandbytes \
    peft --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig, Trainer, DataCollatorForSeq2Seq
import evaluate
import pandas as pd

In [2]:
#copy toj=ken from https://huggingface.co/settings/tokens
import huggingface_hub
hf_token = 'hf_...'
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

In [3]:
!huggingface-cli whoami

aapalireza


## Download Training Dataset

## 📘 Dataset Reference

**Title:** [Enhancing English-Persian Neural Machine Translation with a Large-Scale Parallel Dataset and Relative Position Representations](https://ieeexplore.ieee.org/abstract/document/10967409)  
**Authors:** Alireza Kamyab, Negar Baghaei Nejad, [Alireza Akhavanpour](https://class.vision/teacher/%D8%B9%D9%84%DB%8C%D8%B1%D8%B6%D8%A7-%D8%A7%D8%AE%D9%88%D8%A7%D9%86-%D9%BE%D9%88%D8%B1/)  
**Affiliation:** [Shenasa AI](https://shenasa.ai), Tehran, Iran  
**Dataset on HuggingFace:** [shenasa/English-Persian-Parallel-Dataset](https://huggingface.co/datasets/shenasa/English-Persian-Parallel-Dataset)


In [4]:
huggingface_dataset_name = "shenasa/English-Persian-Parallel-Dataset"
dataset = load_dataset(huggingface_dataset_name)

In [5]:
dataset = dataset['train']
dataset

Dataset({
    features: ['flash fire .', 'فلاش آتش .'],
    num_rows: 3960172
})

In [6]:
import pandas as pd
from datasets import Dataset


source_lang_col = 'src_lang'
target_lang_col = 'tgt_lang'

df_train = dataset.to_pandas()
df_train.columns = ['src_lang', 'tgt_lang']

dataset = Dataset.from_pandas(df_train)
dataset

Dataset({
    features: ['src_lang', 'tgt_lang'],
    num_rows: 3960172
})

### Keep only examples where the target translation is a valid string,
#### filtering out any records with missing or non-string translations.

In [7]:
dataset = dataset.filter(lambda x: isinstance(x[target_lang_col], str))
dataset = dataset.filter(lambda x: isinstance(x[source_lang_col], str))
dataset

Filter:   0%|          | 0/3960172 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3960159 [00:00<?, ? examples/s]

Dataset({
    features: ['src_lang', 'tgt_lang'],
    num_rows: 3960158
})

In [8]:
source_lang_col = dataset.column_names[0]
target_lang_col = dataset.column_names[1]
for i in range(0,100,20):
    print(f"Source: {dataset[i][source_lang_col]}")
    print(f"Target: {dataset[i][target_lang_col]}")
    print("-" * 50)

Source: superheats the air . burns the lungs like rice paper .
Target: هوا را فوق العاده گرم می کند . ریه ها را مثل کاغذ برنج می سوزاند .
--------------------------------------------------
Source: you gave elena a chance .
Target: تو به النا فرصت دادی
--------------------------------------------------
Source: chris . chris , come here .
Target: کریس . کریس بیا اینجا
--------------------------------------------------
Source: l cant find an access hatch . baby , look at me . look at me .
Target: من نمی توانم دریچه دسترسی پیدا کنم . عزیزم به من نگاه کن به من نگاه کن .
--------------------------------------------------
Source: look , baby , the waters gon na rise , and l am right here , okay .
Target: نگاه کن عزیزم ، آبها بالا می آیند ، و من اینجا هستم ، باشه .
--------------------------------------------------


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

model_name = "google/gemma-2-9b-it"

bnbConfig = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    quantization_config=bnbConfig
)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [10]:
def print_number_of_trainable_model_parameters(model):
    all_model_params = model.num_parameters()
    trainable_model_params = sum(param.numel() for param in model.parameters() if param.requires_grad)

    percentage_trainable = 100 * trainable_model_params / all_model_params if all_model_params > 0 else 0

    return (f"Trainable model parameters: {trainable_model_params}\n"
            f"All model parameters: {all_model_params}\n"
            f"Percentage of trainable model parameters: {percentage_trainable:.2f}%")

print(print_number_of_trainable_model_parameters(model))

Trainable model parameters: 918109696
All model parameters: 9241705984
Percentage of trainable model parameters: 9.93%


In [11]:
index = 200

source_lang = dataset[index][source_lang_col]
target_lang = dataset[index][target_lang_col]

prompt = f"""
Translate the following English text to Persian:
English: {source_lang}

Persian translation:
"""
print(prompt)



Translate the following English text to Persian:
English: direct upwards first door to the left .

Persian translation:



In [12]:
device = torch.device("cuda")
inputs = tokenizer(prompt, return_tensors='pt').to(device)

In [13]:
inputs

{'input_ids': tensor([[     2,    108,  49688,    573,   2412,   4645,   2793,    577,  48848,
         235292,    108,  12023, 235292,   3280,  45509,   1370,   4444,    577,
            573,   2731,    954,    109, 146233,  17183, 235292,    108]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]], device='cuda:0')}

In [14]:
model = model.eval()
with torch.no_grad():
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=100,
      )[0],
      skip_special_tokens=True
  )
print(output)


Translate the following English text to Persian:
English: direct upwards first door to the left .

Persian translation:
در ورودی اول را به سمت چپ و سپس به سمت بالا مستقیم بروید.






In [15]:
index = 95000

source_lang = dataset[index][source_lang_col]
target_lang = dataset[index][target_lang_col]

prompt = f"""
Translate the following English text to Persian:
English: {source_lang}

Persian translation:
"""

device = torch.device("cuda")
inputs = tokenizer(prompt, return_tensors='pt').to(device)
with torch.no_grad():
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=100,
          do_sample=False
      )[0],
      skip_special_tokens=True
  )

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE Translation:\n{target_lang}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Translate the following English text to Persian:
English: Sweat had broken on his forehead like dew.

Persian translation:

---------------------------------------------------------------------------------------------------
BASELINE Translation:
عرق مثل شبنم روی پیشانی اش نشسته بود .

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:

Translate the following English text to Persian:
English: Sweat had broken on his forehead like dew.

Persian translation:
عرق بر پیشانی اش مانند قطرات شبنم ریخته بود.






## Lets Load **WIKI PEPC** as a Benchmark!

## 📘 PEPC: Parallel English-Persian Corpus

**عنوان:** [PEPC: Parallel English-Persian Corpus Extracted from Wikipedia](https://iasbs.ac.ir/~ansari/nlp/pepc.html)  
**نویسندگان:** Akbar Karimi, Ebrahim Ansari, Bahram Sadeghi Bigham  
**سازمان:** Institute for Advanced Studies in Basic Sciences (IASBS), Zanjan, Iran  
**دسترسی به دیتاست:** [دانلود PEPC](https://iasbs.ac.ir/~ansari/nlp/pepc.html)


In [17]:
!wget https://iasbs.ac.ir/~ansari/nlp/files_pepc/AK_Test_1K.rar

--2025-04-30 04:40:21--  https://iasbs.ac.ir/~ansari/nlp/files_pepc/AK_Test_1K.rar
Resolving iasbs.ac.ir (iasbs.ac.ir)... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


85.185.211.14, 37.152.161.100
connected. to iasbs.ac.ir (iasbs.ac.ir)|85.185.211.14|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 101715 (99K) [application/x-rar-compressed]
Saving to: ‘AK_Test_1K.rar’

AK_Test_1K.rar      100%[===================>]  99.33K   186KB/s    in 0.5s    

2025-04-30 04:40:23 (186 KB/s) - ‘AK_Test_1K.rar’ saved [101715/101715]



In [18]:
!apt-get install unrar

Reading package lists... 0%

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  unrar
0 upgraded, 1 newly installed, 0 to remove and 84 not upgraded.
Need to get 146 kB of archives.
After this operation, 343 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 unrar amd64 1:6.1.5-1ubuntu0.1 [146 kB]
Fetched 146 kB in 1s (105 kB/s) 
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package unrar.
(Reading database ... 40201 files and directories currently installed.)
Preparing to unpack .../unrar_1%3a6.1.5-1ubuntu0.1_amd64.deb ...
Unpacking unrar (1:6.1.5-1ubuntu0.1) ...
Setting up unrar (1:6.1.5-1ubuntu0.1) ...
update-alternatives: using /usr/bin/unrar-nonfree to provide /usr/bin/unrar (unrar) in auto mode
Processing triggers for man-db (2.10.2-1) ...


In [19]:
!unrar x AK_Test_1K.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from AK_Test_1K.rar

Extracting  ak-test-1k.fa                                             OK 
Extracting  ak-test-1k.en                                             OK 
All OK


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
!ls

 AK_Test_1K.rar			   ak-test-1k.en   onstart.sh
'NMT_with_Gemma2_9b(1)(1).ipynb'   ak-test-1k.fa   ports.log


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Evaluate on **WIKI PEPC**


In [17]:
def load_text(path):
    with open(path, 'r', encoding='utf-8') as f:
        texts = f.readlines()
    texts = [line.strip() for line in texts if line.strip()]
    return texts


source_texts = load_text('ak-test-1k.en')
reference_texts = load_text('ak-test-1k.fa')

len(source_texts), len(reference_texts)

(1000, 1000)

In [18]:
source_texts = source_texts[:100]
reference_texts = reference_texts[:100]
len(source_texts), len(reference_texts)

(100, 100)

In [19]:
from tqdm.notebook import tqdm

bleu = evaluate.load("bleu")

predictions = []
references = []
source_texts_list = []

MAX_LENGTH = 512

with torch.no_grad():
    for i in tqdm(range(len(source_texts)), desc="Processing translations"):
        source_text = source_texts[i]
        reference_text = reference_texts[i]

        prompt = f"""
        Translate the following English text to Persian:
        English: {source_text}

        Persian translation:"""

        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_LENGTH
        ).to(device)

        input_token_length = inputs["input_ids"].shape[1]

        full_output_ids = model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
            do_sample=False  # deterministic
        )[0]

        output_only_ids = full_output_ids[input_token_length:]
        cleaned_output = tokenizer.decode(
            output_only_ids,
            skip_special_tokens=True
        )
        prediction_text = cleaned_output.strip()

        predictions.append(prediction_text)
        references.append([reference_text])
        source_texts_list.append(source_text)

results = bleu.compute(predictions=predictions, references=references)

print(f"BLEU Score: {results['bleu']:.4f}")

Processing translations:   0%|          | 0/100 [00:00<?, ?it/s]

BLEU Score: 0.0737


In [20]:
df_samples = pd.DataFrame({
    'Source (English)': source_texts_list[:10],
    'Reference (Persian)': [ref[0] for ref in references[:10]],
    'Prediction (Gemma)': predictions[:10]
})

df_samples

,Source (English),Reference (Persian),Prediction (Gemma)
0,Politics without the imagination is bureaucrac...,سياست بدون تخیل، چيزي نيست به جز كاغذ بازي، ول...,سیاست بدون تصور، بروکراسی است، اما تصور هرگز ی...
1,The shantytowns built on the outskirts of Lima...,زاغه‌هايي كه در حومه‌هاي شهر ليما، پايتخت پرو ...,
2,Populated by underemployed laborers from the c...,اين شهرك‌ها مملوند از كارگراني بي كار که از لي...,
3,a number of these shantytowns were originally ...,بسياري از اين زاغه‌ها در دهه هفتاد به وجود آمد...,
4,"In 2002, one of these shantytowns, named Venta...","در سال 2002 ، يكي از اين زاغه‌ها به نام ""ونتان...",در سال ۲۰۰۲، یکی از این محله‌های فقیرنشین به ن...
5,The surrounding landscape is desert.,پيرامون اين شهر چيزي به جز صحرا نيست,منظره پیرامون صحراست.
6,"To move a mountain on this landscape, Francis ...","براي جا به جا كردن آن ""فرانسيس آليس""، به هر يك...",برای جابجایی یک کوه در این منظره، فرنسیس آلیس ...
7,or as he described it in an interview publishe...,يا به گفته خود آليس در مصاحبه‌اش با مجله آرت ف...,یا همانطور که او در مصاحبه‌ای که در مجله آرتفو...
8,This human comb pushed a certain quantity of s...,اين شانه انساني توانست مقدار زيادي شن را جابجا...,یک شانه انسانی، مقدار معینی از ماسه را به مساف...
9,This combination of poetic vagueness and preci...,اين تركيب مبهم شاعرانه همراه با دستورعملي دقيق...,این ترکیب از ابهام شاعری و دستورالعمل دقیق، هس...


# peft

In [21]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [22]:
modules

['gate_proj', 'q_proj', 'k_proj', 'o_proj', 'v_proj', 'down_proj', 'up_proj']

### Prepares the model for training with low-bit precision (e.g., 4-bit or 8-bit) to reduce memory usage.

In [23]:
from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)

In [24]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=16,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)
peft_model = get_peft_model(model, peft_config)
print(print_number_of_trainable_model_parameters(peft_model))

Trainable model parameters: 54018048
All model parameters: 9295724032
Percentage of trainable model parameters: 0.58%


In [30]:
#dataset = dataset.shuffle(seed=65).select(range(100000))

In [25]:
def preprocess_fn(examples):
    prompts = [
        f"Translate the following English text to Persian:\n"
        f"English: {src}\n\nPersian translation:"
        for src in examples[source_lang_col]
    ]
    targets = examples[target_lang_col]
    full_texts = [p + " " + t for p, t in zip(prompts, targets)]
    
    model_inputs = tokenizer(
        full_texts,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    labels = model_inputs["input_ids"].clone()
    
    for i, prompt in enumerate(prompts):
        prompt_ids = tokenizer(prompt, truncation=True, max_length=128)["input_ids"]
        prompt_len = len(prompt_ids)
        labels[i][:prompt_len] = -100  # mask prompt from loss

    model_inputs["labels"] = labels
    return model_inputs


In [26]:
tokenized_train = dataset.map(
    preprocess_fn,
    batched=True,
    remove_columns=[source_lang_col, target_lang_col]
)

Map:   0%|          | 0/3960158 [00:00<?, ? examples/s]

In [27]:
split_dataset = tokenized_train.train_test_split(test_size=0.2, seed=42)
tokenized_train = split_dataset["train"]
tokenized_eval = split_dataset["test"]

In [28]:
output_dir = f'./peft-english_to_persian_gemma2-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    gradient_accumulation_steps=2,
    warmup_steps =1000,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    # fp16=True,
    learning_rate=2e-5,
    logging_steps=500,
    eval_steps=1000,
    max_steps=3000,
    label_names=["labels"],
    log_level="info",
    report_to="none",
)

### Enable gradient checkpointing for the original model to save memory during training
#### by trading off some computational overhead. This is especially useful for large models.

In [42]:
#peft_training_args.gradient_checkpointing = True

In [37]:
#peft_model.gradient_checkpointing_enable()

In [29]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding="max_length",
    max_length=64,
    return_tensors="pt"
)

In [30]:
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,  # مجموعه اعتبارسنجی
    data_collator=data_collator,
    tokenizer=tokenizer
)

/tmp/ipykernel_3415/869292460.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
train_output = peft_trainer.train()

***** Running training *****
  Num examples = 3,168,126
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 3,000
  Number of trainable parameters = 54,018,048
It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/venv/main/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for mo

Step,Training Loss
500,2.432300
1000,0.811200
1500,0.707000
2000,0.670800
2500,0.655400


Saving model checkpoint to ./peft-english_to_persian_gemma2-1745993061/checkpoint-500
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-9b-it/snapshots/11c9b309abf73637e4b6f9a3fa1e92e615547819/config.json
Model config Gemma2Config {
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "gemma2",
  "num_attention_heads": 16,
  "num_hidden_layers": 42,
  "num_key_value_heads": 8,
  "pad_token_id": 0,
  "query_pre_attn_scalar": 256,
  "rms_norm_eps": 1e-06,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "

In [ ]:
peft_model_path="./peft-english_to_persian_gemma2"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)